<h1> Import libraries here </h1>

In [1]:
import sklearn as sk
import numpy as np
import pandas as pd
import math 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder

## For Data visualisation 
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sn
# from adspy_shared_utilities import plot_class_regions_for_classifier_subplot #Save to git first


<h2>1. DATA PROCESSING</h2>

<h3>Functions</h3>

In [2]:
def split(x, y, train = 0.8, test = 0): 
    valid = 1 - train - test
    x_trainy, x_valid, y_trainy, y_valid = train_test_split(x, y, train_size = (train+test), test_size = valid)
    if test > 0:
        x_train, x_test, y_train, y_test = (
            train_test_split(x_trainy, y_trainy, train_size = train/(train+test), test_size = test/(train+test)))
    else:
        x_train, y_train =  x_trainy, y_trainy
        x_train, y_train =  x_trainy, y_trainy
        x_test, y_test = 0, 0
    return x_train, x_valid, x_test, y_train, y_valid, y_test

def score_RandomForest(x_train, x_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100)
    model.fit(x_train, y_train)
    preds = model.predict(x_valid)
    return model, mean_absolute_error(y_valid, preds)

## Keep getting different variables with the highest importance??
def LinReg (X, y, function, var):
    feature_list, importances, feature_importances = function
    sorted_by_second = sorted(feature_importances, key=lambda tup: tup[1], reverse=True)
    ## Edit ind_var to only use indepent variables 
    ind_var = var
    X = X[ind_var]
    regr = linear_model.LinearRegression()
    regr.fit(X,y)
    return regr

def feature_importance(model, x_train):
    feature_list = list(x_train.columns)
    importances = model.feature_importances_
    feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
    return feature_list, importances, feature_importances


<h3> Preprocessing </h3>

In [3]:
dataPure = pd.read_excel("Digital_Data.xlsx")
data = dataPure.copy()
data = data.replace(to_replace = 0, value = np.NaN)
label_encoder = LabelEncoder()
s = (data.dtypes == 'object')
cat_cols = list(s[s].index)
cat_cols.remove('Region')
for col in cat_cols:
    data[col] = label_encoder.fit_transform(data[col].astype(str))

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_data = pd.DataFrame(OH_encoder.fit_transform(np.array(data['Region']).reshape(-1,1)))

print()
# One-hot encoding removed index; put it back
OH_cols_data.index = data.index

# Remove categorical columns (will replace with one-hot encoding)
num_data = data.drop(['Region'], axis=1)

# Add one-hot encoded columns to numerical features
data = pd.concat([num_data, OH_cols_data], axis=1)

print(data)

##Dropping Features HERE
data = data.drop(["Project Ref No.", "Blade length (m)", "Tower height (m)","Built duration"], axis = 1)

data_sets = [data.drop(["Single Blade Weight (te)"], axis = 1), data.drop(["Nacelle Weights"], axis = 1)]
X = []
Y = []
for i in range(2):
    data_sets[i] = data_sets[i].dropna()
    x = data_sets[i].drop(columns = data_sets[i].columns[4])
    y = data_sets[i][data_sets[i].columns[4]]
    x_train, x_valid, x_test, y_train, y_valid, y_test = split(x,y)
    X.append([x_train, x_valid, x_test])
    Y.append([y_train, y_valid, y_test])


    Project Ref No.  Operator  Water depth   a1  Turbine rating (kW)  \
0              1085         0            0  250               4000.0   
1              1086         0            0  250               4000.0   
2              1087         0            0  250                450.0   
3              1088         0            0  240               2300.0   
4              1089         1            0  240               2450.0   
..              ...       ...          ...  ...                  ...   
61             1146         0            2   57                530.0   
62             1147         0            2   45               1100.0   
63             1148         0            2   45               1160.0   
64             1149         0            2   40                320.0   
65             1150         0            2   30               1000.0   

    Blade length (m)  Tower height (m)  Nacelle Weights  \
0               69.0             207.0             90.0   
1               

<h3>Random Forrest</h3>

In [4]:
model = []
for i in range(2):
    m, mae = score_RandomForest(X[i][0], X[i][1], Y[i][0], Y[i][1])
    model.append(m)
    print(mae)

11.40344
4.043833333333332


<h3>Lin Regression and Independent Variables</h3>


In [5]:
feature_import = []
reg = []
for i in range(2):
    ## Identify the features of importance from Random Forest
    feature_import.append(feature_importance(model[i], X[i][0]))
    ## Multivariable Linear Regression
    take_forward = feature_import[i][1] > 0.1
    reg.append(LinReg(x, y, feature_import[i], np.array(feature_import[i][0])[take_forward]))
    print(feature_import[i][2])
    print(np.array(feature_import[i][0])[take_forward])
    print(reg[i].coef_)
    print(reg[i].intercept_)
    print('\n')

[('Operator', 0.02), ('Water depth', 0.02), ('a1', 0.34), ('Turbine rating (kW)', 0.41), ('Type', 0.14), ('Metocean', 0.01), (0, 0.02), (1, 0.01), (2, 0.01), (3, 0.02), (4, 0.0)]
['a1' 'Turbine rating (kW)' 'Type']
[ 0.03647248  0.00024014 -0.12845157]
9.834225970505777


[('Operator', 0.0), ('Water depth', 0.22), ('a1', 0.22), ('Turbine rating (kW)', 0.25), ('Type', 0.1), ('Metocean', 0.06), (0, 0.02), (1, 0.03), (2, 0.04), (3, 0.01), (4, 0.04)]
['Water depth' 'a1' 'Turbine rating (kW)' 'Type']
[-3.10093221e+00  9.01530483e-03 -1.83698761e-04  1.64884044e-01]
17.034904886148148


